In [ ]:
import process_copus
import numpy as np
import glob
import os
import matplotlib
import matplotlib.pyplot as plt
import scipy.io
import math
import sys
%matplotlib inline

In [ ]:
matplotlib.__version__

In [ ]:
nb_id = 1738668234
import time
try:
    print(nb_id)
except NameError:
    print(round(time.time()))

In [ ]:
def out_path(filename):
    global outdir
    path = os.path.join(outdir, filename)
    return path

In [ ]:
def get_i_from_path(filepath):
    filename = os.path.basename(filepath)
    root, ext = os.path.splitext(filename)
    last_part = root.split('_')[-1]
    index = int(last_part)
    return index

In [ ]:
def remove_prefix(text, prefix):
    if text.startswith(prefix):
        return text[len(prefix):]
    else:
        raise ValueError

In [ ]:
def get_i2_from_path(filepath):
    filename = os.path.basename(filepath)
    root, ext = os.path.splitext(filename)
    index_str = remove_prefix(root, 'm_full')
    index = int(index_str)
    return index

In [ ]:
def get_txt_filename(path_raw):
    path = os.path.abspath(path_raw)
    root, ext = os.path.splitext(path)
    txt_filename = root + '.txt'
    return txt_filename

In [ ]:
def get_outdir(filepath_raw):
    filepath = os.path.abspath(filepath_raw)
    parts = filepath.split(os.sep)
    outdir = os.path.join(*parts[-3:])
    return outdir

In [ ]:
class MyInfo:
    # Give names of class members.
    def __repr__(self):
        return self.__class__.__name__ + '(' + str(list(self.__dict__.keys())) + ')'
    def __str__(self):
        return self.__class__.__name__ + '(' + str(list(self.__dict__.keys())) + ')'

In [ ]:
GHz = 1e9
ps = 1e-12
nm = 1e-9
um = 1e-6

In [ ]:
dirpath_raw = '../yig_stripline_x_out/06_yig/yig_isofreq_019.out/'
dirpath = os.path.abspath(dirpath_raw)
parent_dir, root = os.path.split(dirpath)
subtitle_prefix = os.path.join(os.path.basename(parent_dir), root)
this_i = get_i_from_path(dirpath)
outdir = get_outdir(dirpath)

In [ ]:
os.listdir(parent_dir)

In [ ]:
if not os.path.isdir(outdir):
    os.makedirs(outdir, exist_ok=True)

In [ ]:
info = MyInfo()
info.txt_path = get_txt_filename(dirpath)
with open(info.txt_path) as fp:
    params = process_copus.parse_logfile(fp.readlines())
info.freq = float(params['f'])
info.Nx = int(params['Nx'])
info.Ny = int(params['Ny'])
info.dx = params['c']
info.dy = params['c']
info.xpos = np.linspace(0, info.Nx, info.Nx)*info.dx
info.ypos = np.linspace(0, info.Ny, info.Ny)*info.dy
info.tstep = float(params['tstep'])
info.maxdt = float(params['maxdt'])
info.params = params

In [ ]:
%%time
M1 = []
index = []
filenames = []
filename_prefix = []
for filepath in glob.glob(os.path.join(dirpath, 'm_full*.npy')):
    M1_raw = np.load(filepath)
    M1.append(M1_raw.squeeze())
    index.append(get_i2_from_path(filepath))
    this_filename = os.path.basename(filepath)
    filenames.append(this_filename)
    this_prefix, this_ext = os.path.splitext(this_filename)
    filename_prefix.append(this_prefix)

In [ ]:
INDEX = -1 % len(M1) # constrain to be positive
print(M1[INDEX][2].min(), M1[INDEX][2].max())
THIS_T = INDEX*info.tstep
fig, ax = plt.subplots(
    constrained_layout=True,
    figsize=(2*6.4, 2*4.8)
)
norm = matplotlib.colors.TwoSlopeNorm(vmin=-2e0, vcenter=0, vmax=2e0)
pcm = ax.pcolormesh(
    info.xpos/um,
    info.ypos/um,
    M1[INDEX][2],
    cmap='bwr',
#     vmin=-20,
#     vmax=-100,
    norm = norm,
    shading='nearest',
#     norm=matplotlib.colors.LogNorm(),
);
xmid = (info.xpos.min() + info.xpos.max())/2
ymid = (info.ypos.min() + info.ypos.max())/2
dx = 2
dy = 2
# ax.set_xlim(xmid/um - dx, xmid/um + dx)
# ax.set_ylim(ymid/um - dy, ymid/um + dy)
ax.set_xlabel('x [um]')
ax.set_ylabel('y [um]')
fig.colorbar(mappable=pcm, ax=ax, label="$M_z$ [A/m]")
ax.set_aspect('equal')
ax.set_title(subtitle_prefix + ", t = {:.1f} ps, maxdt = {} ps, nb_id = {}".format(THIS_T/ps, info.maxdt/ps, nb_id), fontsize=10);
fig.suptitle("$M_z$, f = {:.1f} GHz".format(info.freq/GHz));

In [ ]:
fig.canvas.draw()

In [ ]:
plt.close(fig); del(fig, ax)

In [ ]:
INDEX = -1 % len(M1) # constrain to be positive
THIS_T = INDEX*info.tstep
fig, ax = plt.subplots(
    constrained_layout=True,
    figsize=(2*6.4, 2*4.8)
)
blackwhite_cm = matplotlib.colors.LinearSegmentedColormap.from_list('black_white', ['black', 'white'], N=2)
pcm = ax.pcolormesh(
    info.xpos/um,
    info.ypos/um,
    np.sign(M1[INDEX][2]),
    cmap=blackwhite_cm,
    shading='nearest',
#     norm=matplotlib.colors.LogNorm(),
);
xmid = (info.xpos.min() + info.xpos.max())/2
ymid = (info.ypos.min() + info.ypos.max())/2
dx = 2
dy = 2
# ax.set_xlim(xmid/um - dx, xmid/um + dx)
# ax.set_ylim(ymid/um - dy, ymid/um + dy)
ax.set_xlabel('x [um]')
ax.set_ylabel('y [um]')
fig.colorbar(mappable=pcm, ax=ax, label="$M_z$ [A/m]")
ax.set_aspect('equal')
ax.set_title(subtitle_prefix + ", t = {:.1f} ps, maxdt = {} ps, nb_id = {}".format(THIS_T/ps, info.maxdt/ps, nb_id), fontsize=10);
fig.suptitle("$M_z$, f = {:.1f} GHz".format(info.freq/GHz));

In [ ]:
fig.canvas.draw()

In [ ]:
plt.close(fig); del(fig, ax)

In [ ]:
info.xpos.shape, info.ypos.shape, M1[0].shape

In [ ]:
%%time
%%capture
# continuous color scale
debug = True
to_plot = list(enumerate(M1))
norm = matplotlib.colors.TwoSlopeNorm(vmin=-2e-1, vcenter=0, vmax=2e-1)
for (n, this_M1) in to_plot:
    if debug:
        print(n)
    fig, ax = plt.subplots(
        constrained_layout=True,
        figsize=(2*6.4, 2*4.8),
    )
    pcm = ax.pcolormesh(
        info.xpos/um,
        info.ypos/um,
        this_M1[2],
        cmap='bwr',
        norm = norm,
        shading='nearest',
    )
    ax.set_aspect('equal')
    fig.colorbar(mappable=pcm, ax=ax, label="$M_z$ [A/m]", location='right')
    this_t = n*info.tstep
    ax.set_title(
        "i = {}, t = {:.1f} ps, {}x{}".format(n, this_t/ps, info.Nx, info.Ny),
    )
    ax.set_xlabel('x position [um]')
    ax.set_ylabel('y position [um]')
    fig.suptitle("$M_z$, f = {:.1f} GHz".format(info.freq/GHz));
    fig.savefig(
        out_path('Mz_heatmap_' + filename_prefix[n] + ".png"),
        bbox_inches='tight',
        facecolor="w",
        dpi=300,
    );
    plt.close(fig); del fig, ax

In [ ]:
%%time
%%capture
# black and white, sign only
debug = True
to_plot = list(enumerate(M1))
blackwhite_cm = matplotlib.colors.LinearSegmentedColormap.from_list('black_white', ['black', 'white'], N=2)
for (n, this_M1) in to_plot:
    if debug:
        print(n)
    fig, ax = plt.subplots(
        constrained_layout=True,
        figsize=(2*6.4, 2*4.8),
    )
    pcm = ax.pcolormesh(
        info.xpos/um,
        info.ypos/um,
        np.sign(this_M1[2]),
        cmap=blackwhite_cm,
        shading='nearest',
    )
    ax.set_aspect('equal')
    fig.colorbar(mappable=pcm, ax=ax, label="$M_z$ [A/m]", location='right')
    this_t = n*info.tstep
    ax.set_title(
        "i = {}, t = {:.1f} ps, {}x{}".format(n, this_t/ps, info.Nx, info.Ny),
    )
    ax.set_xlabel('x position [um]')
    ax.set_ylabel('y position [um]')
    fig.suptitle("$M_z$, f = {:.1f} GHz".format(info.freq/GHz));
    fig.savefig(
        out_path('Mz_bw_heatmap_' + filename_prefix[n] + ".png"),
        bbox_inches='tight',
        facecolor="w",
        dpi=300,
    );
    plt.close(fig); del fig, ax

In [ ]:
%%time
n_cols = 4
to_plot = list(enumerate(M1))
n_rows = math.ceil(len(to_plot)/n_cols)
if n_rows > 30:
    print("too many rows: {} > 30".format(n_rows))
    sys.exit(1)
norm = matplotlib.colors.TwoSlopeNorm(vmin=-2e-0, vcenter=0, vmax=2e-0)
fig, axs = plt.subplots(nrows=n_rows, ncols=n_cols, constrained_layout=True, figsize=(10*0.3*n_cols, 9*0.3*n_rows), dpi=200)
plotted = {}
for (n, this_M1) in to_plot:
    i, j = divmod(n, n_cols)
    print(n, i, j)
    this_t = n*info.tstep
    if n_rows == 1:
        raise NotImplementedError
    else:
        pcm = axs[i][j].pcolormesh(
            info.xpos,
            info.ypos,
            this_M1[2],
            cmap='bwr',
            norm = norm,
            shading='nearest',
        )
        axs[i][j].set_title(
            "i = {}, t = {:.1f} ps".format(n, this_t/ps),
        )
        axs[i][j].set_aspect('equal')
        axs[i][j].axis('off')
        plotted[(i, j)] = True
if n_rows == 1:
    cbar = fig.colorbar(mappable=pcm, ax=axs[:], location='right',
                 # shrink=0.5,
                )
else:
    cbar = fig.colorbar(mappable=pcm, ax=axs[:, :], location='right',
                 # shrink=0.5,
                )
ticklabels = cbar.ax.get_yticklabels()
cbar.ax.set_yticklabels(ticklabels, fontsize=14)
for i in range(n_rows):
    for j in range(n_cols):
        if (i, j) not in plotted:
            if n_rows == 1:
                raise NotImplementedError
            else:
                axs[i][j].remove()
# fig.supxlabel("x [um]", fontsize=14)
# fig.supylabel("y [um]", fontsize=14);
fig.suptitle("$M_z$, f = {:.1f} GHz".format(info.freq/GHz), fontsize=14);

In [ ]:
# fig.canvas.draw()

In [ ]:
%%time
fig.savefig(
    out_path("grid_all_Mz_heatmap.png"),
    bbox_inches='tight',
    facecolor="w",
    dpi=300,
);

In [ ]:
plt.close(fig); del(fig, axs)

In [ ]:
%%time
n_cols = 4
to_plot = list(enumerate(M1))
n_rows = math.ceil(len(to_plot)/n_cols)
if n_rows > 30:
    print("too many rows: {} > 30".format(n_rows))
    sys.exit(1)
blackwhite_cm = matplotlib.colors.LinearSegmentedColormap.from_list('black_white', ['black', 'white'], N=2)
fig, axs = plt.subplots(nrows=n_rows, ncols=n_cols, constrained_layout=True, figsize=(10*0.3*n_cols, 9*0.3*n_rows), dpi=200)
plotted = {}
for (n, this_M1) in to_plot:
    i, j = divmod(n, n_cols)
    print(n, i, j)
    this_t = n*info.tstep
    if n_rows == 1:
        raise NotImplementedError
    else:
        pcm = axs[i][j].pcolormesh(
            info.xpos,
            info.ypos,
            np.sign(this_M1[2]),
            cmap=blackwhite_cm,
            shading='nearest',
        )
        axs[i][j].set_title(
            "i = {}, t = {:.1f} ps".format(n, this_t/ps),
        )
        axs[i][j].set_aspect('equal')
        axs[i][j].axis('off')
        plotted[(i, j)] = True
if n_rows == 1:
    cbar = fig.colorbar(mappable=pcm, ax=axs[:], location='right',
                 # shrink=0.5,
                )
else:
    cbar = fig.colorbar(mappable=pcm, ax=axs[:, :], location='right',
                 # shrink=0.5,
                )
ticklabels = cbar.ax.get_yticklabels()
cbar.ax.set_yticklabels(ticklabels, fontsize=14)
for i in range(n_rows):
    for j in range(n_cols):
        if (i, j) not in plotted:
            if n_rows == 1:
                raise NotImplementedError
            else:
                axs[i][j].remove()
# fig.supxlabel("x [um]", fontsize=14)
# fig.supylabel("y [um]", fontsize=14);
fig.suptitle("$M_z$, f = {:.1f} GHz".format(info.freq/GHz), fontsize=14);

In [ ]:
# fig.canvas.draw()

In [ ]:
%%time
fig.savefig(
    out_path("grid_all_Mz_heatmap_bw.png"),
    bbox_inches='tight',
    facecolor="w",
    dpi=300,
);

In [ ]:
plt.close(fig); del(fig, axs)